In [1]:
import pickle
import sys
import os
from Securities import Security
# Add the parent directory (my_project) to the Python path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('boltmarket-main'), '..')))
from importlib import reload
import Tradables
reload(Tradables)
from Tradables import Underlier, MarketTable
from Securities import Security, get_security
from gs_quant.session import GsSession
import pandas as pd
import os
import yfinance as yf
from datetime import datetime, timedelta



####################################################################
# FINANCEPY BETA Version 0.290 - This build:  10 May 2023 at 18:52 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



C:\Users\jacob\AppData\Local\Programs\Python\Python311\Lib\site-packages\financepy\products\equity\equity_binomial_tree.py:36: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [2]:
date_offsets = {
    '1D': timedelta(days=1),
    '1W': timedelta(weeks=1),
    '2W': timedelta(weeks=2),
    '3W': timedelta(weeks=3),
    '1M': timedelta(days=30),
    '2M': timedelta(days=60),
    '3M': timedelta(days=90),
    '6M': timedelta(days=180),
    '9M': timedelta(days=270),
    '1Y': timedelta(days=365),
    'Overnight': timedelta(days=1),
    'Tomorrow Next': timedelta(days=1),
    'Spot Next': timedelta(days=2),
    'One Week': timedelta(weeks=1),
    'Two Weeks': timedelta(weeks=2),
    'Three Weeks': timedelta(weeks=3),
    'One Month': timedelta(days=30),
    'Two Months': timedelta(days=60),
    'Three Months': timedelta(days=90),
    'Four Months': timedelta(days=120),
    'Five Months': timedelta(days=150),
    'Six Months': timedelta(days=180),
    'Seven Months': timedelta(days=210),
    'Eight Months': timedelta(days=240),
    'Nine Months': timedelta(days=270),
    'Ten Months': timedelta(days=300),
    'Eleven Months': timedelta(days=330),
    'One Year': timedelta(days=365),
    'Two Years': timedelta(days=730),
    'Three Years': timedelta(days=1095),
    'Four Years': timedelta(days=1460),
    'Five Years': timedelta(days=1825),
    'Six Years': timedelta(days=2190),
    'Seven Years': timedelta(days=2555),
    'Ten Years': timedelta(days=3650),
}


"""use excel wings to load surface from EURUSD Risk&Marks"""
guipath = os.path.abspath('C:\\Users\\jacob\\bolt-hub\\GUI.xlsx')
marks = pd.read_excel(guipath, sheet_name="EURUSD Risks&Marks")



def save_security(sec):
    """save security to pickle in the securities folder"""
    with open(f"Securities/{sec.name}.pkl", "wb") as f:
        pickle.dump(sec, f)

def get_intraday_weighting():
    ticker_symbol = 'EURUSD=X'
    start_date = '2022-01-01'
    end_date = '2023-06-01'
    # Get the data from Yahoo Finance
    data = yf.download(ticker_symbol, start=start_date, end=end_date, interval="1h")

    # Extract the open and close prices by hour
    hourly_data = data[['Open', 'Close']].resample('H').last()

    """create a column representing hour of the day"""
    hourly_data['Hour'] = hourly_data.index.hour
    """create a column representing change in price from previous hour"""
    hourly_data['Change'] = abs(hourly_data['Open']-hourly_data['Close'])/hourly_data['Open']

    """group by hour and calculate mean change"""
    hourly_data = hourly_data.groupby('Hour')['Change'].mean()
    return hourly_data

def extract_forward_curve(marks):
    """extract forward curve"""
    fwd = marks.drop(index=range(5))
    last_row_index = fwd.index[-31]
    # Select all rows up to and including the last 31st row, and assign it back to the dataframe
    fwd = fwd.iloc[:last_row_index+1, :]
    fwd = fwd.iloc[:, :4]
    fwd.iloc[1:, 0] = [date_offsets[tenor] for tenor in fwd.iloc[1:, 0]]
    # get the values of the top row as a series
    new_columns = fwd.iloc[0]
    # assign the new column names to the dataframe
    fwd.columns = new_columns
    # drop the first row since it's no longer needed as column names
    fwd = fwd.iloc[1:]
    # set the index to the values in the first column by position
    fwd = fwd.set_index(fwd.columns[0])
    # drop the first column since it's now the index
    fwd = fwd.drop(columns=[fwd.columns[0]])
    fwd = fwd.rename_axis(index=None).rename_axis(columns=None)
    fwd.drop('Bid', axis=1, inplace=True)
    return fwd

def extract_vol_surface(marks):
    """extract vol surface"""
    vols = marks.drop(index=range(33))
    vols = vols.iloc[0:, :18]
    # get the values of the top row as a series
    new_columns = [5,10,15,20,25,30,35,40,45,50,-40,-35,-30,-25,-20,-15,-10,-5]
    # assign the new column names to the dataframe
    vols.columns = new_columns
    # drop the first row since it's no longer needed as column names
    vols = vols.iloc[1:]
    # set the index to the values in the first column by position
    vols = vols.set_index(vols.columns[0])
    # drop the first column since it's now the index
    vols = vols.drop(columns=[vols.columns[0]])
    vols = vols.rename_axis(index=None).rename_axis(columns=None)
    new_index = [date_offsets[tenor] for tenor in vols.index]
    vols.set_index(pd.Index(new_index), inplace=True)
    return vols


In [3]:
eurusd = Underlier()
eurusd.load_forward_curve(extract_forward_curve(marks))
eurusd.load_vol_surface(extract_vol_surface(marks))
eurusd.mark_spot(1.0935)
eurusd.load_intraday_weights(get_intraday_weighting())

sec = Security('EURUSD', eurusd)
sec.save()

[*********************100%***********************]  1 of 1 completed


In [ ]:
usdjpy = Underlier()
usdjpy.load_forward_curve(extract_forward_curve(marks))
usdjpy.load_vol_surface(extract_vol_surface(marks))
usdjpy.mark_spot(145)
usdjpy.load_intraday_weights(get_intraday_weighting())

sec = Security('USDJPY', usdjpy)
sec.save()


In [35]:
products = MarketTable(pd.DataFrame())
Security('Tradables', products).save()

In [ ]:
sec = pickle.load(open('EURUSD.pkl', 'rb'))

In [ ]:
sys.path.append('C:\\Users\\jacob\\bolt-hub\\boltmarket-main\\Securities')


In [13]:
import requests
from bs4 import BeautifulSoup

def scrape_website(url):
    try:
        # Add headers to the request to mimic a regular browser request
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        # Send an HTTP GET request to the URL with headers
        response = requests.get(url, headers=headers)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find the elements containing the articles' titles and links
            article_titles = soup.find_all('h2')  # You may need to adjust the tag based on the website's structure
            article_links = soup.find_all('a')  # You may need to adjust the tag based on the website's structure

            # Extract and print the titles and links
            for title, link in zip(article_titles, article_links):
                print("Title:", title.text.strip())
                print("Link:", link['href'])
                print("-" * 50)
        else:
            print("Failed to retrieve the page. Status code:", response.status_code)
    except requests.RequestException as e:
        print("Error while sending the request:", e)



if __name__ == "__main__":
    url = "https://www.cmegroup.com/trading/fx/cme-fx-options-vol-converter.html"  # Replace with the actual URL of the website you want to scrape
    scrape_website(url)


KeyboardInterrupt: 

In [49]:
import yfinance as yf
import pandas as pd

def get_hourly_eurusd_data():
    # Define the ticker symbol for EUR/USD
    ticker = 'EURUSD=X'

    # Define the start and end date for the data
    start_date = '2021-09-01'
    end_date = '2023-08-01'

    # Fetch daily EUR/USD data from Yahoo Finance
    data = yf.download(ticker, start=start_date, end=end_date, progress=False, interval='1h')
    return data

data = get_hourly_eurusd_data()

In [50]:
data['change'] = abs(data['Close'] - data['Open'])
data['hour'] = data.index.hour
by_hour = data.groupby('hour')['change'].mean()


In [60]:
(by_hour.cumsum()/by_hour.cumsum().max()).to_csv('intraday_weighting.csv')
